# Debug sentence classification


### Labels

In [1]:
import numpy as np
import pickle as pkl
from tqdm import tqdm, trange
from ftfy import fix_text
from collections import defaultdict
import torch, io, gzip, json, random, argparse, os
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import (BertTokenizer, BertConfig, AdamW, BertForSequenceClassification,
        WarmupLinearSchedule)
from ftfy import fix_text
from arxiv_public_data.config import DIR_BASE, DIR_OUTPUT, DIR_FULLTEXT
f_metadata = os.path.join(DIR_BASE, 'arxiv-metadata-oai-2019-03-01.json.gz')

#Got these from Matt
cat_map = {
  "astro-ph": "astro-ph",
  "cond-mat": "cond-mat",
  "cs": "cs",
  "gr-qc": "gr-qc",
  "hep-ex": "hep-ex",
  "hep-lat": "hep-lat",
  "hep-ph": "hep-ph",
  "hep-th": "hep-th",
  "math-ph": "math-ph",
  "nlin": "nlin",
  "nucl-ex": "nucl-ex",
  "nucl-th": "nucl-th",
  "physics": "physics",
  "quant-ph": "quant-ph",
  "math": "math",
  "q-bio": "q-bio",
  "q-fin": "q-fin",
  "stat": "stat",
  "eess": "eess",
  "econ": "econ",
  "acc-phys": "physics.acc-ph",
  "adap-org": "nlin.AO",
  "alg-geom": "math.AG",
  "ao-sci": "physics.ao-ph",
  "atom-ph": "physics.atom-ph",
  "bayes-an": "physics.data-an",
  "chao-dyn": "nlin.CD",
  "chem-ph": "physics.chem-ph",
  "cmp-lg": "cs.CL",
  "comp-gas": "nlin.CG",
  "dg-ga": "math.DG",
  "funct-an": "math.FA",
  "mtrl-th": "cond-mat.mtrl-sci",
  "patt-sol": "nlin.PS",
  "plasm-ph": "physics.plasm-ph",
  "q-alg": "math.QA",
  "solv-int": "nlin.SI",
  "supr-con": "cond-mat.supr-con"
}


# I should experiment with and without this
def clean_doc(x):
    return fix_text(x)


def load_data(N, fname, data_type):
    
    
    #MAX_LENS = [50, 250, 500]  #truncate all titles, abstracts, fulltext to this level
    #N, data_type = args.N, args.data_type
    #if data_type == 'title':
    #     MAX_LEN = MAX_LENS[0]
    #elif data_type == 'abstract':
    #     MAX_LEN = MAX_LENS[1]
    #elif data_type == 'fulltext':
    #     MAX_LEN = MAX_LENS[2]
    
    MAX_LEN = 512  #BERT default
    input_ids = []
    labels, label_dict, ctr = [], {}, 0
    attention_masks = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    with gzip.open(fname, 'rt', encoding='utf-8') as fin:
        for row in fin.readlines():

            #Load metadata
            m = json.loads(row)

            #Build label list
            if data_type != 'fulltext':
                sentence = clean_doc(m[data_type])
            else:
                sentence = load_ith_fulltext(i)  ###needs to be filled in
                sentence = clean_doc(sentence)

            # We need to add special tokens at the beginning and end of each sentence for BERT to work properly
            sentence = "[CLS] " + sentence + " [SEP]" 

            #category
            category = m['categories'][0].split(' ')[0]

            #update cateogies -- apply matt's map
            if category in cat_map: category = cat_map[category]

            #Then add to the dics
            if category not in label_dict:
                index = len(label_dict)
                label_dict[category] = index  # ex: {'hep-ph':0, 'math.CO:1',,}
            else:
                index = label_dict[category]
            labels.append(index)


            #Tokenize
            tokenized_sentence = tokenizer.tokenize(sentence)  #Ex: ['the', 'cat', 'ate']

            #Convert to IDs + pad
            input_id = tokenizer.convert_tokens_to_ids(tokenized_sentence)  #Ex: [1,10,3]
            input_id = pad_sequences([input_id], maxlen=MAX_LEN, dtype="long",truncating="post",padding="post")
            input_ids.append(input_id[0])
            
            #Attention mask
            seq_mask = [float(i>0) for i in input_id[0]]
            attention_masks.append(seq_mask)
            
            #Ctr
            ctr += 1
            if ctr >= N: break
                
    return np.array(input_ids), attention_masks, labels, label_dict  



#N, data_type = 10**7, 'title'
#metadata = load_data(N,f_metadata)
#sentences, labels, label_dict_new = process_data_sub(metadata, data_type=data_type)
#len(label_dict_new)

I1210 15:47:19.839966 140225670215488 file_utils.py:39] PyTorch version 1.3.1 available.
I1210 15:47:19.862981 140225670215488 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [25]:
N, data_type = 10**7, 'title'
metadata = load_data(N,f_metadata)
sentences, labels, label_dict_old = process_data(metadata, data_type=data_type)
len(label_dict_old)

171

In [22]:
cat_map

{'astro-ph': 'astro-ph',
 'cond-mat': 'cond-mat',
 'cs': 'cs',
 'gr-qc': 'gr-qc',
 'hep-ex': 'hep-ex',
 'hep-lat': 'hep-lat',
 'hep-ph': 'hep-ph',
 'hep-th': 'hep-th',
 'math-ph': 'math-ph',
 'nlin': 'nlin',
 'nucl-ex': 'nucl-ex',
 'nucl-th': 'nucl-th',
 'physics': 'physics',
 'quant-ph': 'quant-ph',
 'math': 'math',
 'q-bio': 'q-bio',
 'q-fin': 'q-fin',
 'stat': 'stat',
 'eess': 'eess',
 'econ': 'econ',
 'acc-phys': 'physics.acc-ph',
 'adap-org': 'nlin.AO',
 'alg-geom': 'math.AG',
 'ao-sci': 'physics.ao-ph',
 'atom-ph': 'physics.atom-ph',
 'bayes-an': 'physics.data-an',
 'chao-dyn': 'nlin.CD',
 'chem-ph': 'physics.chem-ph',
 'cmp-lg': 'cs.CL',
 'comp-gas': 'nlin.CG',
 'dg-ga': 'math.DG',
 'funct-an': 'math.FA',
 'mtrl-th': 'cond-mat.mtrl-sci',
 'patt-sol': 'nlin.PS',
 'plasm-ph': 'physics.plasm-ph',
 'q-alg': 'math.QA',
 'solv-int': 'nlin.SI',
 'supr-con': 'cond-mat.supr-con'}

In [28]:
'adap-org' in label_dict_old, 'adap-org' in label_dict_new

(True, False)

Looks good.

### Truncation

When we truncate the titles, do we chop off the special " [SEP] " token?

In [5]:
N, data_type = 10**2, 'title'
metadata = load_data(N,f_metadata)
sentences, labels, label_dict = process_data_sub(metadata, data_type=data_type)
print('Num classes = {}'.format(len(label_dict)))
print('Tokenizing')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

I1205 16:47:50.308459 139851126974272 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/khev/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Num classes = 43
Tokenizing


In [10]:
MAX_LEN = 512  # BERT pretrained model width

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts] #bert tokenizer
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post") #pad

In [12]:
input_ids[:,0]

array([101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101])

In [14]:
tokenizer.convert_ids_to_tokens([101])

['[CLS]']

Conclusion: the pretrained BERT used MAX_LEN = 512, so we're stuck with this